In [1]:
!pip install evidently

In [2]:
import pandas as pd
import numpy as np
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

import warnings
warnings.filterwarnings('ignore')

In [3]:
my_json = pd.read_json('s3://576118004139-parameterizing-master/Master.json')
l2_bucket = my_json.loc["l2-bucket"]["buckets"]

In [4]:
for month_year in [202501]: #change pred month
    ref_data = pd.read_csv(f's3://576118004139-ap-south-1-kbl-datamart-l2/datamartl2/ib_usage_model_phase1/output_files/output_data_202307.csv')
    curr_data = pd.read_csv(f's3://576118004139-ap-south-1-kbl-datamart-l2/datamartl2/ib_usage_model_phase1/output_files/output_data_202502.csv')
    details = pd.read_csv(f's3://576118004139-ap-south-1-kbl-datamart-l2/datamartl2/ib_usage_model_phase1/output_files/train_202502.csv')
    metric1 = pd.read_csv(f's3://576118004139-ap-south-1-kbl-datamart-l2/ib_usage_model_phase1/output_files/metrics_m_202502.csv')
    display(metric1.head())

    numeric_cols = ['last_M12_AVG_SBA', 'TOTAL_DISTINCT_PRODS',
           'tot_ci_neft_cr_amt_l6m', 'tot_ci_imps_cr_amt_l6m',
           'Occupation_SALARIED', 'tot_ci_db_high_tckt_amt_l3m', 'age', 'vintage',
           'REGION_NAME_BANGALORE', 'tot_ci_ktk_atm_tran_l12m',
           'tot_ci_pos_amt_l12m', 'tot_branch_induced_db_tran_l12m',
           'mandates_tran_l12m']

    def calculate_psi(expected, actual, buckettype='bins', buckets=10, axis=0):

        def psi(expected_array, actual_array, buckets):

            def scale_range (input, min, max):
                input += -(np.min(input))
                input /= np.max(input) / (max - min)
                input += min
                return input


            breakpoints = np.arange(0, buckets + 1) / (buckets) * 100

            if buckettype == 'bins':
                breakpoints = scale_range(breakpoints, np.min(expected_array), np.max(expected_array))
            elif buckettype == 'quantiles':
                breakpoints = np.stack([np.percentile(expected_array, b) for b in breakpoints])



            expected_percents = np.histogram(expected_array, breakpoints)[0] / len(expected_array)
            actual_percents = np.histogram(actual_array, breakpoints)[0] / len(actual_array)

            def sub_psi(e_perc, a_perc):

                if a_perc == 0:
                    a_perc = 0.0001
                if e_perc == 0:
                    e_perc = 0.0001

                value = (e_perc - a_perc) * np.log(e_perc / a_perc)
                return(value)

            psi_value = np.sum(sub_psi(expected_percents[i], actual_percents[i]) for i in range(0, len(expected_percents)))

            return(psi_value)

        if len(expected.shape) == 1:
            psi_values = np.empty(len(expected.shape))
        else:
            psi_values = np.empty(expected.shape[axis])

        for i in range(0, len(psi_values)):
            if len(psi_values) == 1:
                psi_values = psi(expected, actual, buckets)
            elif axis == 0:
                psi_values[i] = psi(expected[:,i], actual[:,i], buckets)
            elif axis == 1:
                psi_values[i] = psi(expected[i,:], actual[i,:], buckets)

        return(psi_values)

    results = []

    for col in numeric_cols:
        val = calculate_psi(ref_data[col], curr_data[col])
        drift_status = ""
        if val < 0.1:
            drift_status = "No significant drift/shift detected"
        elif 0.1 <= val < 0.2:
            drift_status = "Slight drift/shift detected"
        else:
            drift_status = "Significant drift/shift detected"

        results.append({'Variable_name': col, 'CSI': val, 'drift_status': drift_status})

    # Convert results to DataFrame
    
    results_df = pd.DataFrame(results)
    results_df.head()

#     results_df = pd.DataFrame(results)
    results_df.to_html(f'ib_outputs/csi_{month_year}.html') 
    
    data_drift_report = Report(metrics=[DataDriftPreset()])
    data_drift_report.run(current_data = curr_data, reference_data= ref_data, column_mapping=None)
    data_drift_report.save_html(f"ib_outputs/file_{month_year}.html")
#     display(data_drift_report)
    details.to_csv(f'ib_outputs/captureRate_{month_year}.csv', index=None)

Month_Year  Accuracy    recall       AUC      Gini        KS
0      202502  0.711151  0.834301  0.711151  0.690791  0.542396

In [5]:
curr_data.shape

(682807, 14)

In [6]:
 results_df

Variable_name       CSI  \
0                  last_M12_AVG_SBA  0.003021   
1              TOTAL_DISTINCT_PRODS  0.002150   
2            tot_ci_neft_cr_amt_l6m  0.001113   
3            tot_ci_imps_cr_amt_l6m  0.001388   
4               Occupation_SALARIED  0.001520   
5       tot_ci_db_high_tckt_amt_l3m  0.001112   
6                               age  0.004372   
7                           vintage  0.008027   
8             REGION_NAME_BANGALORE  0.000008   
9          tot_ci_ktk_atm_tran_l12m  0.007478   
10              tot_ci_pos_amt_l12m  0.001654   
11  tot_branch_induced_db_tran_l12m  0.034010   
12               mandates_tran_l12m  0.029497   

                           drift_status  
0   No significant drift/shift detected  
1   No significant drift/shift detected  
2   No significant drift/shift detected  
3   No significant drift/shift detected  
4   No significant drift/shift detected  
5   No significant drift/shift detected  
6   No significant drift/shift detected  
7   No significant drift/shift detected  
8   No significant drift/shift detected  
9   No significant drift/shift detected  
10  No significant drift/shift detected  
11  No significant drift/shift detected  
12  No significant drift/shift detected

In [18]:
#results_df.to_csv(f'ib_outputs/csi_{month_year}.csv')

In [7]:
for month_year in [202501]: #month change
#     ref_data = pd.read_csv('s3://l2-karnatakabank-bucket/ib_usage_model_phase1/output_files/output_data_202307.csv')
#     curr_data = pd.read_csv(f's3://l2-karnatakabank-bucket/ib_usage_model_phase1/output_files/output_data_{month_year}.csv')
    ref_data = pd.read_csv(f's3://576118004139-ap-south-1-kbl-datamart-l2/datamartl2/ib_usage_model_phase1/output_files/leads_data_202307.csv')
    curr_data = pd.read_csv(f's3://576118004139-ap-south-1-kbl-datamart-l2/datamartl2/ib_usage_model_phase1/output_files/leads_data_202502.csv')

    numeric_cols_prob = ['Probabilities']

    def calculate_psi(expected, actual, buckettype='bins', buckets=10, axis=0):

        def psi(expected_array, actual_array, buckets):

            def scale_range (input, min, max):
                input += -(np.min(input))
                input /= np.max(input) / (max - min)
                input += min
                return input


            breakpoints = np.arange(0, buckets + 1) / (buckets) * 100

            if buckettype == 'bins':
                breakpoints = scale_range(breakpoints, np.min(expected_array), np.max(expected_array))
            elif buckettype == 'quantiles':
                breakpoints = np.stack([np.percentile(expected_array, b) for b in breakpoints])



            expected_percents = np.histogram(expected_array, breakpoints)[0] / len(expected_array)
            actual_percents = np.histogram(actual_array, breakpoints)[0] / len(actual_array)

            def sub_psi(e_perc, a_perc):

                if a_perc == 0:
                    a_perc = 0.0001
                if e_perc == 0:
                    e_perc = 0.0001

                value = (e_perc - a_perc) * np.log(e_perc / a_perc)
                return(value)

            psi_value = np.sum(sub_psi(expected_percents[i], actual_percents[i]) for i in range(0, len(expected_percents)))

            return(psi_value)

        if len(expected.shape) == 1:
            psi_values = np.empty(len(expected.shape))
        else:
            psi_values = np.empty(expected.shape[axis])

        for i in range(0, len(psi_values)):
            if len(psi_values) == 1:
                psi_values = psi(expected, actual, buckets)
            elif axis == 0:
                psi_values[i] = psi(expected[:,i], actual[:,i], buckets)
            elif axis == 1:
                psi_values[i] = psi(expected[i,:], actual[i,:], buckets)

        return(psi_values)


    results = []

    for col in numeric_cols_prob:
        val = calculate_psi(ref_data[col], curr_data[col])
        drift_status = ""
        if val < 0.1:
            drift_status = "No significant drift/shift detected"
        elif 0.1 <= val < 0.2:
            drift_status = "Slight drift/shift detected"
        else:
            drift_status = "Significant drift/shift detected"

        results.append({'Variable_name': col, 'PSI': val, 'drift_status': drift_status})

    # Convert results to DataFrame

    results_df = pd.DataFrame(results)

results_df.to_html(f'ib_outputs/psi_{month_year}.html')

In [20]:
ref_data = pd.read_csv(f's3://576118004139-ap-south-1-kbl-datamart-l2/datamartl2/ib_usage_model_phase1/output_files/leads_data_202307.csv')
curr_data = pd.read_csv('leads_data_202412.csv')


In [ ]:
# ref_data['EventRate_DF'].value_counts()

In [8]:
results_df

Variable_name      PSI                         drift_status
0  Probabilities  0.02432  No significant drift/shift detected

In [ ]:
# base = pd.read_parquet(f"""s3://576118004139-ap-south-1-kbl-datamart-l2/datamartl2/ib_usage_model_phase1/ib_base_data_'202408'/""")

In [36]:
# base.head()

In [37]:
# base['region_name'].value_counts()